In [1]:
import asyncio
import aiohttp
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# -----------------------
# async fetch NASA POWER data
# -----------------------
async def fetch_power(session, lat, lon, start, end, params):
    url = (
        "https://power.larc.nasa.gov/api/temporal/hourly/point"
        f"?start={start}&end={end}&latitude={lat}&longitude={lon}"
        f"&parameters={','.join(params)}&format=JSON&community=RE"
    )
    async with session.get(url) as resp:
        resp.raise_for_status()
        js = await resp.json()
        return js["properties"]["parameter"]

async def get_raw_data_async(target_dt, lat, lon, params, window=5, years_back=10):
    dfs = []
    async with aiohttp.ClientSession() as session:
        tasks = []
        for y in range(1, years_back+1):
            past_year = target_dt.year - y
            start = (target_dt.replace(year=past_year) - timedelta(days=window)).strftime("%Y%m%d")
            end   = (target_dt.replace(year=past_year) + timedelta(days=window)).strftime("%Y%m%d")
            tasks.append(fetch_power(session, lat, lon, start, end, params))
        results = await asyncio.gather(*tasks)

    # convert to dataframe list
    for i, r in enumerate(results, 1):
        past_year = target_dt.year - i
        df = pd.DataFrame({p: pd.Series(r[p]) for p in params})
        print(df)
        df.index = pd.to_datetime(df.index, format="%Y%m%d%H")
        df = df.resample("12H").mean()
        print(df)
        df["year"] = past_year
        df["hour"] = df.index.hour
        dfs.append(df)

    full_df = pd.concat(dfs, axis=0)
    return full_df.reset_index(drop=True)

# -----------------------
# Create hourly–yearly averages
# -----------------------
def make_hourly_yearly_avg_df(raw_df, params):
    avg_df = raw_df.groupby(["year", "hour"])[params].mean().reset_index()
    return avg_df

# -----------------------
# Example usage
# -----------------------
params = ["T2M","RH2M","PRECTOTCORR","ALLSKY_SFC_SW_DWN","WS2M"]
target_dt = datetime(2025,10,2,12)

# In Colab, use await directly instead of asyncio.run() because an event loop is already running.
raw_df = await get_raw_data_async(target_dt, 10.75, 106.67, params)
print("Raw data (head):")
print(raw_df.head())
print(raw_df.tail())


avg_df = make_hourly_yearly_avg_df(raw_df, params)
print("\nYearly-hourly average (head):")
print(avg_df.head())
print(avg_df.tail())

              T2M   RH2M  PRECTOTCORR  ALLSKY_SFC_SW_DWN  WS2M
2024092700  24.89  97.55         3.55                0.0  1.01
2024092701  24.70  97.74         2.62                0.0  0.79
2024092702  24.54  97.72         1.11                0.0  0.67
2024092703  24.44  97.48         1.04                0.0  0.62
2024092704  24.34  97.26         1.23                0.0  0.62
...           ...    ...          ...                ...   ...
2024100719  26.99  91.49        15.18                0.0  0.60
2024100720  26.68  92.96        59.31                0.0  0.61
2024100721  26.38  94.39       129.12                0.0  0.62
2024100722  26.04  95.81       126.07                0.0  0.62
2024100723  25.75  96.84        68.84                0.0  0.64

[264 rows x 5 columns]
                           T2M       RH2M  PRECTOTCORR  ALLSKY_SFC_SW_DWN  \
2024-09-27 00:00:00  26.039167  91.389167     3.406667         240.745833   
2024-09-27 12:00:00  27.256667  87.226667     3.358333         189

C:\Users\DELL\AppData\Local\Temp\ipykernel_14852\516164146.py:38: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample("12H").mean()
C:\Users\DELL\AppData\Local\Temp\ipykernel_14852\516164146.py:38: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample("12H").mean()
C:\Users\DELL\AppData\Local\Temp\ipykernel_14852\516164146.py:38: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample("12H").mean()
C:\Users\DELL\AppData\Local\Temp\ipykernel_14852\516164146.py:38: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample("12H").mean()
C:\Users\DELL\AppData\Local\Temp\ipykernel_14852\516164146.py:38: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample("12H").mean()
C:\Users\DELL\A